In [30]:
import numpy as np
import pandas as pd
from collections import defaultdict
import re
from datetime import datetime

print('Hello Insight.')
print('Start processing.')

def valid_date(trans_dt): # Should not assume the year field holds any particular value. So the year value should be checked.
    
    valid = 1
    
    if not len(trans_dt)==8:
        valid = 0
    else:
        if int(trans_dt[4:8])>2018 or int(trans_dt[4:8])<2013:
            valid = 0
        if int(trans_dt[0:1])>12 or int(trans_dt[0:1])<1:
            valid = 0
        if int(trans_dt[2:3])>31 or int(trans_dt[2:3])<1:
            valid = 0
    
    return valid


def valid_zip(zipcode):
    
    valid = 1
    
    if len(zipcode)<5:
        valid = 0
    
    return valid

def valid_name(name):
    
    valid = 1
    
    if len(name)<1:
        valid = 0
    
    return valid

# CMTE_ID, AMT, OTHER_ID assumed to be easily verified.

def valid_all(donation): # Include all the invalid checks.
    
    valid = 1
    
    CMTE_ID = donation[0]
    #print(CMTE_ID)
    NAME = donation[7]
    #print(NAME)
    ZIP_CODE = donation[10]
    #print(ZIP_CODE)
    #DONAR = donation[7] + '_' + donation[10][0:5]
    TRANS_DT = donation[13]
    #print(TRANS_DT)
    #YEAR = donation[13][4:8]
    TRANS_AM = donation[14]
    #print(TRANS_AM)
    OTHER = donation[15]
    #print(OTHER)
    
    if len(CMTE_ID)==0:
        valid = 0
    if len(TRANS_AM)==0 or int(TRANS_AM)<0:
        valid = 0
    if len(OTHER)>0:
        valid = 0
    
    if not valid_date(TRANS_DT):
        valid = 0
    if not valid_zip(ZIP_CODE):
        valid = 0
    if not valid_name(NAME):
        valid = 0
    
    #print(valid_date(TRANS_DT))
    #print(valid_zip(ZIP_CODE))
    #print(valid_name(NAME))
    
    return valid

def order_time(donors_val):    # Delete the first donor and return all the repeat donor in a list.
    donors_ordered = []
    years = set()

    for donor_val in donors_val:
        years.add(int(donor_val['YEAR']))

    year_first = min(years)

    for donor_val in donors_val:
        if int(donor_val['YEAR'])>year_first:
            donors_ordered.append(donor_val)

    return donors_ordered

Hello Insight.
Start processing.


In [31]:
if __name__ == '__main__':
    #NUM = 10 # Num of lines to test on
    #i = 0

    hash_donars = defaultdict(list)
    #hash_zips = defaultdict(list)
    zips = defaultdict(set)

    with open('./insight_testsuite/tests/test_1/input/itcont.txt') as f:

        try:

            while True:
                line = next(f)
                donation = line.split('|')

                if valid_all(donation):
                    #pass
                #else:
                    #print(i)
                    #print(line)
                    donation_dict = dict()
                    donation_dict['CMTE_ID'] = donation[0]
                    donation_dict['NAME'] = donation[7]
                    donation_dict['ZIP_CODE'] = donation[10][0:5]
                    donation_dict['DONAR'] = donation[7] + '_' + donation[10][0:5]
                    donation_dict['TRANS_DT'] = donation[13]
                    donation_dict['YEAR'] = donation[13][4:8]
                    donation_dict['TRANS_AM'] = donation[14]
                    # dona['OTHER_ID'] = donation[15]

                    #i += 1
                    #print(dona)

                    hash_donars[donation_dict['DONAR']].append(donation_dict)
                    #hash_zips[dona['ZIP_CODE']].append(dona)
                    zips[donation_dict['ZIP_CODE']].add(donation[7] + '_' + donation[10][0:5])

        except StopIteration:

            pass

    #print(hash_donars)
    #print(hash_zips)
    #print(zips)

    with open('./output.txt', 'a') as out:

        # Zip - Donars - Donors

        for zip_id in zips: # For each zip code
            i = 0
            amount = 0
            donars_zip= zips[zip_id]

            for donars in donars_zip: # For each donars inside each zip code

                if len(hash_donars[donars])>1:
                  #  pass
                #else:
                    donors_ordered = order_time(hash_donars[donars])    # Only return the donors with repeat years. NO first year.
                    #print(donors_ordered)
                    #i = 1
                    for donors in donors_ordered:  # For each repeat donor inside each donars
                        #print(donors)

                        i += 1
                        amount += int(donors['TRANS_AM'])

                        repeat_donor = '|'.join([donors['CMTE_ID'],donors['ZIP_CODE'],donors['TRANS_DT'],str(amount),str(i)])

                        print(repeat_donor)
                        out.write(repeat_donor)
                        out.write('\n')


    #print(zips)
    #print(hash_donars)


C00409409|56003|10222014|500|1
C00545921|56003|10292014|3100|2
C00042366|55987|10222014|350|1
C00247569|22066|11142014|624|1
C00473652|22066|10312014|874|2
C00473652|22066|12312014|1374|3
C00081356|22066|10172014|2374|4
C00112896|22066|11242014|2998|5
C00075820|22066|12292014|3248|6
C00075820|22066|10232014|3498|7
C00409581|22066|12112014|4498|8
C00200584|22066|10302014|7098|9
C00436386|22066|12302014|7498|10
C00436386|22066|12302014|8098|11
C00395947|22066|10152014|8306|12
C00395947|22066|10312014|8514|13
C00000935|22066|10172014|9014|14
C00000935|22066|10242014|10014|15
C00003418|22066|10162014|42414|16
C00171330|22066|12122014|42830|17
C00238865|22066|10312014|43330|18
C00551572|22066|10272014|44330|19
C00200584|22066|10302014|46930|20
C00550707|22066|10282014|47530|21
C00457226|22066|11182014|47780|22
C00409409|56001|10302014|500|1
C00409409|56001|10162014|3000|2
C00550707|56001|10202014|3600|3
C00550707|56001|11182014|4184|4
C00550707|56001|11182014|5599|5
C00002881|22203|11132014

C00060103|99508|11142014|2520|8
C00060103|99508|12312014|2728|9
C00060103|99508|11282014|2936|10
C00060103|99508|12152014|3144|11
C00068528|92708|11242014|416|1
C00068528|92708|12312014|1248|2
C00224691|92708|10172014|2248|3
C00005660|74104|11212014|250|1
C00012880|59901|10162014|1000|1
C00382424|46804|10212014|250|1
C00544221|25301|10132014|500|1
C00495028|25301|10232014|5500|2
C00506774|25301|12182014|8100|3
C00550012|85012|10212014|500|1
C00196246|85012|10202014|865|2
C00445981|85012|10312014|1065|3
C00445981|85012|12312014|1465|4
C00010470|70808|11242014|226|1
C00010470|70808|12312014|679|2
C00543983|70808|11242014|1679|3
C00543983|70808|10242014|4279|4
C00325126|70808|10152014|4519|5
C00451336|70808|10232014|5019|6
C00283135|70808|12232014|5269|7
C00283135|70808|11242014|5769|8
C00045781|70808|11252014|10769|9
C00010603|70808|10132014|11019|10
C00128876|70808|10152014|12019|11
C00040998|97501|11212014|2500|1
C00196246|97501|12292014|2865|2
C00421982|10016|10292014|250|1
C00381459|

C00564294|20006|10122014|17208|20
C00010603|20006|12112014|24608|21
C00489658|20006|10202014|34608|22
C00010603|20006|10302014|44608|23
C00042366|20006|10152014|57008|24
C00366401|20006|10232014|59508|25
C00547414|20006|10232014|62008|26
C00473652|20006|10312014|62466|27
C00473652|20006|12312014|62966|28
C00110197|20006|12122014|63586|29
C00542886|20006|10162014|64586|30
C00551648|20006|10172014|65586|31
C00110197|20006|12122014|66186|32
C00156810|20006|10282014|67186|33
C00428052|20006|10312014|68186|34
C00409003|20006|10312014|68686|35
C00491357|20006|10292014|69186|36
C00159954|20006|12312014|69386|37
C00325126|20006|11252014|69736|38
C00110197|20006|12122014|70336|39
C00084491|20006|10152014|70544|40
C00084491|20006|11242014|70960|41
C00084491|20006|12312014|71585|42
C00110197|20006|12122014|71833|43
C00409003|20006|10312014|72333|44
C00330720|20006|10312014|72833|45
C00110197|20006|12122014|73753|46
C00084491|20006|10172014|76253|47
C00148031|20006|10312014|76707|48
C00148031|2000

C00361956|30068|10122014|2950|3
C00077321|30068|10202014|3950|4
C00010603|30068|10272014|4950|5
C00130773|30306|10242014|375|1
C00000935|30306|12142014|400|2
C00401224|30306|12122014|403|3
C00547414|30306|10312014|903|4
C00540443|30306|12192014|1153|5
C00235853|30306|12202014|1453|6
C00009282|30306|11262014|1661|7
C00009282|30306|12152014|1869|8
C00009282|30306|12302014|2077|9
C00009282|30306|10152014|2285|10
C00009282|30306|10312014|2493|11
C00009282|30306|11142014|2701|12
C00009282|20165|11262014|208|1
C00009282|20165|12152014|416|2
C00009282|20165|12302014|624|3
C00009282|20165|10152014|832|4
C00009282|20165|10312014|1040|5
C00009282|20165|11142014|1248|6
C00380303|20165|11212014|1559|7
C00380303|20165|12192014|1766|8
C00032698|20165|12122014|2540|9
C00000935|20165|10272014|3040|10
C00042366|20165|10202014|5040|11
C00009282|62549|11262014|208|1
C00009282|62549|12152014|416|2
C00009282|62549|12302014|624|3
C00009282|62549|10152014|832|4
C00009282|62549|10312014|1040|5
C00009282|62549

C00457622|20009|10272014|43804|35
C00299404|20853|12112014|500|1
C00473652|20853|10312014|700|2
C00473652|20853|12312014|1100|3
C00351379|20853|10312014|2100|4
C00513002|20853|10202014|2600|5
C00551572|20853|12112014|3100|6
C00409979|33767|12232014|3000|1
C00178038|33767|10272014|4850|2
C00550871|33767|12292014|5350|3
C00042622|33767|10282014|10350|4
C00551572|33767|10102014|11350|5
C00551572|33767|10272014|11850|6
C00570416|33714|12172014|500|1
C00002972|65810|11182014|500|1
C00077321|65810|11252014|1000|2
C00128876|33761|10282014|500|1
C00255752|33609|10102014|1000|1
C00501163|33609|10152014|1208|2
C00501163|33609|10312014|1416|3
C00501163|33609|11142014|1624|4
C00501163|33609|11282014|1832|5
C00501163|33609|12152014|2040|6
C00501163|33609|12312014|2248|7
C00250399|33609|12112014|7248|8
C00255752|33609|10112014|8248|9
C00010603|33609|10142014|9248|10
C00042366|33609|10102014|9748|11
C00005561|32303|10202014|250|1
C00546846|32303|10242014|2450|2
C00468959|32303|10212014|2950|3
C005434

C00508341|82718|12312014|1248|3
C00348524|82718|11122014|2248|4
C00510958|82718|10202014|4848|5
C00343459|82718|12122014|5098|6
C00348524|82718|12112014|6098|7
C00002972|82718|12112014|6598|8
C00075820|77007|10272014|500|1
C00543983|77007|11212014|3100|2
C00331439|77007|10152014|4100|3
C00366401|77007|10142014|5100|4
C00458059|77007|10142014|6100|5
C00549592|77007|10142014|7100|6
C00325126|77007|10142014|8100|7
C00541417|77007|10142014|9100|8
C00329896|77007|10142014|10100|9
C00003418|77007|11252014|10600|10
C00003418|77007|10242014|11100|11
C00010603|77007|10272014|11300|12
C00543983|77007|10292014|12450|13
C00543983|77007|10292014|15050|14
C00066472|08628|10312014|5000|1
C00066472|13413|12312014|500|1
C00066472|13413|12152014|1100|2
C00066472|13413|12182014|1600|3
C00066472|13413|11242014|2100|4
C00243659|50323|10172014|290|1
C00003418|50323|10152014|540|2
C00003418|50323|12182014|790|3
C00243659|50323|10172014|1065|4
C00243659|50323|10172014|1365|5
C00545616|50323|10172014|2365|6
C0

C00106146|55114|12292014|599|2
C00106146|55114|10152014|1119|3
C00106146|55114|10152014|1779|4
C00106146|55114|12292014|2549|5
C00476374|55114|12162014|5149|6
C00476374|55114|12162014|7749|7
C00555722|55114|10272014|10349|8
C00253187|55114|10302014|12949|9
C00106146|55114|10152014|13179|10
C00106146|55114|12292014|13448|11
C00028860|08558|12312014|260|1
C00003764|08062|11242014|5000|1
C00106146|08270|10242014|227|1
C00038604|71075|12312014|225|1
C00512236|82717|10162014|2600|1
C00106146|56573|10152014|300|1
C00002972|50071|10202014|500|1
C00471565|78026|11242014|200|1
C00158881|57451|11242014|833|1
C00158881|57451|12312014|1249|2
C00106146|20736|12292014|425|1
C00027466|20736|10312014|1750|2
C00002972|20736|10242014|2250|3
C00002469|20736|12172014|2450|4
C00002972|57226|11182014|500|1
C00409409|56073|10232014|200|1
C00330696|56073|11202014|420|2
C00010868|61704|10152014|1000|1
C00437756|61704|10262014|3600|2
C00458877|61704|10272014|4600|3
C00002972|50219|12242014|500|1
C00548420|67601

C00113803|07002|10302014|250|1
C00028860|53051|12312014|350|1
C00409516|53051|12292014|5350|2
C00409516|53051|12292014|9350|3
C00438747|63304|11252014|200|1
C00219642|63304|11242014|700|2
C00219642|63304|12312014|950|3
C00376343|63304|12242014|1150|4
C00226928|07701|10292014|250|1
C00077321|07701|11242014|500|2
C00077321|07701|11242014|750|3
C00074096|29928|12312014|1250|1
C00027532|29928|11262014|1450|2
C00285254|29928|12212014|2450|3
C00540997|29928|11162014|2490|4
C00540997|29928|11162014|2570|5
C00487470|29928|12162014|3570|6
C00543983|29928|10222014|4570|7
C00547570|29928|10272014|5570|8
C00547570|29928|10272014|6570|9
C00285254|29928|12312014|7570|10
C00410084|84098|12302014|1000|1
C00429662|77251|11282014|416|1
C00429662|77251|12312014|832|2
C00429662|77251|10312014|1248|3
C00000729|70433|10102014|250|1
C00221077|70433|10152014|450|2
C00221077|70433|10152014|650|3
C00345306|85015|12192014|500|1
C00362707|95134|10152014|200|1
C00362707|95134|11242014|800|2
C00362707|95134|1231201

C00161141|32246|11142014|1200|6
C00308676|02339|12172014|1000|1
C00509356|92057|11192014|250|1
C00279380|92057|10312014|500|2
C00235739|76177|10202014|208|1
C00235739|76177|11202014|416|2
C00235739|76177|12202014|624|3
C00027466|78735|10152014|2500|1
C00235739|76137|10202014|208|1
C00235739|76137|11202014|416|2
C00235739|76137|12202014|624|3
C00450320|76006|10152014|1000|1
C00321943|92707|10172014|500|1
C00255257|18936|11212014|350|1
C00101105|49230|10302014|500|1
C00028860|53224|12312014|540|1
C00309567|53224|10172014|740|2
C00028860|53224|12312014|1040|3
C00505297|54155|12312014|5000|1
C00197095|53122|10312014|208|1
C00197095|53122|11152014|416|2
C00197095|53122|10152014|624|3
C00197095|53122|11302014|832|4
C00197095|53122|12152014|1040|5
C00197095|53122|12312014|1248|6
C00003418|53122|10102014|1998|7
C00442996|53122|11182014|2398|8
C00075820|53122|10212014|2648|9
C00003418|53122|10212014|3048|10
C00541417|53703|10312014|1500|1
C00019331|53703|10132014|1750|2
C00330894|53402|10282014

C00032797|10103|12122014|969|3
C00032797|10103|12122014|1295|4
C00032797|10103|12122014|1518|5
C00032797|10103|12122014|1775|6
C00032797|10103|12122014|2247|7
C00032797|10103|12122014|2761|8
C00558189|10103|10202014|5361|9
C00032797|10103|12122014|5721|10
C00032797|10103|12122014|6450|11
C00032797|10103|12122014|6707|12
C00032797|10103|12122014|6981|13
C00032797|10103|12122014|7324|14
C00552364|06754|10192014|250|1
C00432260|06754|10192014|750|2
C00477984|07627|12312014|500|1
C00028860|07627|12312014|760|2
C00140061|99208|10252014|1000|1
C00472159|99208|10102014|1250|2
C00007880|99208|12122014|1500|3
C00007880|99208|11172014|1750|4
C00140061|99208|12312014|2000|5
C00028860|53205|12312014|564|1
C00447565|11249|10272014|1000|1
C00237198|11249|12152014|2500|2
C00028860|05346|12312014|260|1
C00416636|90245|12312014|5000|1
C00340224|90245|12122014|5200|2
C00416636|90245|12192014|5700|3
C00546119|37938|10212014|1000|1
C00075820|31833|10222014|300|1
C00311142|30308|12312014|1000|1
C00361956|3

In [19]:
donation_dict = dict()
donation_dict['CMTE_ID'] = donation[0]
donation_dict['NAME'] = donation[7]
donation_dict['ZIP_CODE'] = donation[10][0:5]
donation_dict['DONAR'] = donation[7] + '_' + donation[10][0:5]
donation_dict['TRANS_DT'] = donation[13]
donation_dict['YEAR'] = donation[13][4:8]
donation_dict['TRANS_AM'] = donation[14]

In [20]:
donation_dict

{'CMTE_ID': 'C00003418',
 'DONAR': 'ELFNER, ALBERT MR. III_02108',
 'NAME': 'ELFNER, ALBERT MR. III',
 'TRANS_AM': '250',
 'TRANS_DT': '10282014',
 'YEAR': '2014',
 'ZIP_CODE': '02108'}

In [4]:
hash_donars[donars]

[{'CMTE_ID': 'C00365031',
  'DONAR': 'BUSTAMANTE, BRUCE_99501',
  'NAME': 'BUSTAMANTE, BRUCE',
  'TRANS_AM': '750',
  'TRANS_DT': '04302014',
  'YEAR': '2014',
  'ZIP_CODE': '99501'},
 {'CMTE_ID': 'C00365031',
  'DONAR': 'BUSTAMANTE, BRUCE_99501',
  'NAME': 'BUSTAMANTE, BRUCE',
  'TRANS_AM': '250',
  'TRANS_DT': '',
  'YEAR': '',
  'ZIP_CODE': '99501'}]